In [11]:
import requests
import json
import pandas as pd
import numpy as np
from config import bls_api_key
import matplotlib.pyplot as plt
import scipy.stats as st 
from sklearn import datasets


In [12]:
url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers = {"content-type": "application/json"}

target_series = ["LNS14000012", "LNS14000025", "LNS14000026","LNS14027659","LNS14027660","LNS14027689","LNS14027662",
                 "LNS14000089","LNS14000091","LNS14000093","LNS14024230","LNS14024887"]

parameters = json.dumps({"seriesid": target_series,
             "startyear": "2008",
             "endyear": "2011",
             "registrationkey": bls_api_key})

p = requests.post(url, data=parameters, headers=headers)

json_data = json.loads(p.text)

json_data

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 459,
 'message': [],
 'Results': {'series': [{'seriesID': 'LNS14000012',
    'data': [{'year': '2011',
      'period': 'M12',
      'periodName': 'December',
      'value': '23.3',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M11',
      'periodName': 'November',
      'value': '24.2',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M10',
      'periodName': 'October',
      'value': '24.2',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M09',
      'periodName': 'September',
      'value': '24.4',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M08',
      'periodName': 'August',
      'value': '25.0',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M07',
      'periodName': 'July',
      'value': '24.7',
      'footnotes': [{}]},
     {'year': '2011',
      'period': 'M06',
      'periodName': 'June',
      'value': '24.5',
      'footnotes': [{}]}

In [13]:
years = []
months = []

for data_point in np.arange(len(json_data["Results"]["series"][0]["data"])):
        years.append(json_data["Results"]["series"][0]["data"][data_point]["year"])
        months.append(json_data["Results"]["series"][0]["data"][data_point]["periodName"])

In [14]:
age_groupdf = pd.DataFrame({"Year": years,
                                "Month": months,
                                "LNS14000012" : "",
                                "LNS14000025" : "",
                                "LNS14000026" : "",
                                "LNS14027659" : "",
                                "LNS14027660" : "",
                                "LNS14027689" : "",
                                "LNS14027662" : "",
                                "LNS14024230" : "",
                                "LNS14000093" : "",
                                "LNS14000091" : "",
                                "LNS14024887" : "",
                                "LNS14000089" : ""
                                                   })
age_groupdf.head()

,Year,Month,LNS14000012,LNS14000025,LNS14000026,LNS14027659,LNS14027660,LNS14027689,LNS14027662,LNS14024230,LNS14000093,LNS14000091,LNS14024887
0,2011,December,,,,,,,,,,,
1,2011,November,,,,,,,,,,,
2,2011,October,,,,,,,,,,,
3,2011,September,,,,,,,,,,,
4,2011,August,,,,,,,,,,,


In [15]:
for series in np.arange(len(json_data["Results"]["series"])):
    for data_point in np.arange(len(json_data["Results"]["series"][series]["data"])):
        age_groupdf.loc[data_point, json_data["Results"]["series"][series]["seriesID"]] = json_data["Results"]["series"][series]["data"][data_point]["value"]

In [20]:
age_groupdf.rename(columns={"LNS14000012":"16-19 yrs",
                                "LNS14000025":"20 yrs & over, Men",
                                "LNS14000026":"20 yrs. & over, Women",
                                "LNS14027659":"Less than a High School Diploma, 25 yrs & over",
                                "LNS14027660":"High School Graduates, No College, 25 yrs & over",
                                "LNS14027662":"Bachelor's degree and higher, 25 yrs & over",
                                "LNS14027689":"Some College or Associate Degree, 25 yrs & over",
                                "LNS14024887":"16-24yo",
                                "LNS14024230":"55yo and over",
                                "LNS14000093":"45-54yo",
                                "LNS14000091":"35-44yo",
                                "LNS14000089":"25-34yo"
                                },
                      inplace=True)
age_groupdf.reset_index(drop=True, inplace=True)
age_groupdf.head(5)

,Year,Month,16-19 yrs,"20 yrs & over, Men","20 yrs. & over, Women","Less than a High School Diploma, 25 yrs & over","High School Graduates, No College, 25 yrs & over","Some College or Associate Degree, 25 yrs & over","Bachelor's degree and higher, 25 yrs & over","20 yrs & over, Men","20 yrs & over, Men","20 yrs & over, Men","20 yrs & over, Men",25-34yo
0,2011,December,23.3,8.0,7.8,13.7,8.7,7.6,4.1,6.2,6.5,6.8,16.7,9.3
1,2011,November,24.2,8.2,7.8,12.8,8.8,7.7,4.4,6.3,6.6,7.0,17.0,9.1
2,2011,October,24.2,8.6,7.8,13.5,9.4,8.1,4.3,7.0,6.9,7.2,16.7,9.5
3,2011,September,24.4,8.7,8.2,14.3,9.4,8.5,4.2,6.6,7.1,7.5,17.3,9.5
4,2011,August,25.0,8.7,7.9,14.1,9.3,8.3,4.2,6.6,7.0,7.7,17.4,9.4


In [22]:
"https://api.stlouisfed.org/fred/releases/dates?api_key="{
    "realtime_start": "2008-01-01",
    "realtime_end": "2011-12-31",
    "order_by": "release_date",
    "sort_order": "desc",
    "count": 1129,
    "offset": 0,
    "limit": 1000,
    "release_dates":
    
    
    
    
    

SyntaxError: invalid syntax (<ipython-input-22-fa065b149209>, line 1)